# Prepare

In [ ]:
!nvidia-smi

In [ ]:
import torch
from functorch import vmap
x = torch.randn(3)
y = vmap(torch.sin)(x)
assert torch.allclose(y, x.sin())

In [ ]:
import functorch
!python -m xformers.info

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from torch import autocast

In [ ]:
# scheduler.save_pretrained('scheduler/EulerDiscreteScheduler')

In [ ]:
scheduler = EulerDiscreteScheduler.from_pretrained("scheduler/EulerDiscreteScheduler", subfolder="scheduler")

In [ ]:
# remote model path
# model_id = "runwayml/stable-diffusion-v1-5"

# local model path
model_id = "models/stable-diffusion-v1.5"



# Use the Euler scheduler here instead
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id,scheduler=scheduler, revision="fp16", torch_dtype=torch.float16)
# pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision="fp16", torch_dtype=torch.float16)
pipe = pipe.to(0)
pipe.enable_attention_slicing()
pipe.safety_checker = (lambda images, clip_input: (images, False))
pipe.device

In [ ]:
# save the model locally after initial download
# pipe.save_pretrained('models/stable-diffusion-v1.5')

In [ ]:
!pip install python-slugify

# RUN

In [ ]:
from slugify import slugify
from time import time

# prompt = "low angle 50mm photograph of a majestic bacon-cheesehamburger composed of grilled beef patties and fresh salad and tomato and secret sauce and melted american cheese and sesame seed buns, 4k award winning food photography, trending on artstation"
# prompt = "plateau de fromages frais et affinés 999€"

# prompt = "La religieuse est composée de deux choux posés l'un sur l'autre, dont le chou supérieur, qui est censé représenter la tête, est deux fois plus petit.Les choux sont recouverts de fondant du même parfum que la crème pâtissière, et une crème au beurre généralement parfumée au café ou à la vanille permet de tenir la tête."

# prompt = "Religieuse is a French pastry, 999€"

# prompt = "top view picture, of a michelin star restaurant dish, award winning food photography, 4k food picture, french signature dish, silver cutlery on each side of the plate, ornated porcelain plate with floral patterns and gilded meandros, crystal drinking glass with red wine, everything is resting on a marble table, fresh flower arrangement in an ornate vase, ming vase holding flowers"


# prompt = "masterwork oil on canvas still life painting of a cheeseburger banquet by Caravaggio,masterwork oil on canvas still life painting of a cheeseburger feast by Caravaggio, juicy ground beef patty, sesame seed bun, melted american cheese, tomato and pickle slices, trending on artstation, red drape in the background, flower bouquet in ornate flower pot, silver cutlery, ornate vase"


# prompt = "masterwork oil on canvas still life painting of a cheeseburger feast by Caravaggio, the cheeseburger is composed of juicy ground beef patty sesame seed bun melted american cheese tomato pickle slices by Caravaggio, trending on artstation, red drape in the background and flower bouquet in ornate flower pot and silver cutlery and ornate vase by Caravaggio"

prompt = "masterwork oil on canvas still life painting of a smartphone laying on a table by Caravaggio, smartphone displaying lockscreen by Caravaggio, a amrtphone still life painting by Caravaggio"
# neg_prompt = "blurry, out of focus, frame, framed, new artist, glossy, recent, unknown"
neg_prompt = "frame, blurry"

set_guidance_scale = 7.5

def txt2img(prompt, neg_prompt, n_iterations, display_image, in_num_inference_steps, input_seed):
    
    seed_generator = torch.Generator('cuda').manual_seed(input_seed)
    image = None
    for i in range(n_iterations):
        with autocast('cuda'):
          image = pipe(prompt, negative_prompt=neg_prompt, height=512, width=512,generator=seed_generator, guidance_scale = set_guidance_scale, num_inference_steps = in_num_inference_steps).images[0]
          image.save(f"images/{slugify(prompt)[:100]}_{i}_{time()}.png")
        if display_image:
            display(image)

    # return image

In [ ]:
import math

prime_list = []

for num in range(400, 500):
    if all(num%i!=0 for i in range(2,int(math.sqrt(num))+1)):
        prime_list.append(num)
        
print(len(prime_list))
prime_list = prime_list[:4]

In [ ]:
seed_list = prime_list

In [ ]:
for curr_seed in seed_list:
    image = txt2img(prompt,neg_prompt, 1, True, 25, curr_seed)